# **Project Group 6**

#### **Team Members:**
| Name              | Student ID |
| ----------------- | ---------- |
| Danial Bashir   |   XXXXXX   | 
| Danial Rana   |   XXXXXX   |
| Danny Le  |   XXXXXX   |
| Christian Torhaug |   564355   |

Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.arima.model as sm
import statsmodels.stats.diagnostic as sd
from arch import arch_model
from arch.univariate import EGARCH
from mvgarch.mgarch import DCCGARCH
from mvgarch.ugarch import UGARCH

#### Retrieving Data

In [45]:
# S&P 500
spx = pd.read_csv("S&P_500.csv")
spx['Date'] = pd.to_datetime(spx['Date'])

# Russell 2000
rut = pd.read_csv('Russell_2000.csv')
rut['Date'] = pd.to_datetime(rut['Date'])

Arima

In [ ]:
best_aic = np.inf
best_order = None
best_model = None

for p in range(6):
    for q in range(6):
        try:
            model = sm.ARIMA(gasoline_data['Return'], order=(p, 0, q)).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_order = (p, 0, q)
                best_model = model
        except:
            continue
        
print(best_model.summary())

model = sm.ARIMA(gasoline_data['Return'], order=(4, 0, 4)).fit()

resid = model.resid

Garch

In [ ]:
p_range = range(1, 5)
q_range = range(1, 5)

results = []

for p in p_range:
    for q in q_range:
        try:
            model = arch_model(resid, vol='GARCH', p=p, q=q, dist='normal')
            fitted_model = model.fit(disp='off')
            results.append({
                'p': p,
                'q': q,
                'AIC': fitted_model.aic,
                'BIC': fitted_model.bic
            })
        except Exception as e:
            print(f"Model GARCH({p},{q}) failed: {e}")
            
results_df = pd.DataFrame(results)
results_df.sort_values(by='AIC', inplace=True)
print(results_df.head())

best_aic_model = results_df.loc[results_df['AIC'].idxmin()]
best_bic_model = results_df.loc[results_df['BIC'].idxmin()]

print(f"Best model by AIC: GARCH({int(best_aic_model['p'])},{int(best_aic_model['q'])}) with AIC = {best_aic_model['AIC']:.4f}")
print(f"Best model by BIC: GARCH({int(best_bic_model['p'])},{int(best_bic_model['q'])}) with BIC = {best_bic_model['BIC']:.4f}")


Stationarity Test

In [ ]:
model = arch_model(resid*100, vol='GARCH', p=1, q=2, dist='normal')
model = model.fit(disp='off')

params = model.params
stationarity_sum = params['alpha[1]'] + params['beta[1]'] + params['beta[2]']

print(f"The Stationarity sum is {stationarity_sum}")

Tarch

In [ ]:
max_p = 5 
max_q = 5

results = []

for p in range(1, max_p + 1):
    for q in range(1, max_q + 1):
        try:
            model = arch_model(resid, vol='GARCH', p=p, o=1, q=q, dist='normal').fit(disp='off')
            results.append({
                'p': p, 'q': q,
                'AIC': model.aic,
                'BIC': model.bic
            })
        except Exception as e:
            print(f"TARCH({p},{q}) failed to converge.")


results_df = pd.DataFrame(results)

best_aic_model = results_df.loc[results_df['AIC'].idxmin()]
best_bic_model = results_df.loc[results_df['BIC'].idxmin()]

print(f"Best model by AIC: TARCH({int(best_aic_model['p'])},{int(best_aic_model['q'])}) with AIC = {best_aic_model['AIC']:.4f}")
print(f"Best model by BIC: TARCH({int(best_bic_model['p'])},{int(best_bic_model['q'])}) with BIC = {best_bic_model['BIC']:.4f}")


In [ ]:
combined_data = pd.merge(gasoline_data, brent_data, left_index=True, right_index=True, suffixes=('_Gasoline', '_Brent')).dropna()

# GARCH with Brent Data as exogenous variable

model = arch_model(combined_data['Return_Gasoline'],  vol='GARCH', p=1, q=1, x=combined_data['Return_Brent'], dist='normal')
garchx_result = model.fit(disp='off')

#print(garchx_result.summary())
fitted_volatility = garchx_result.conditional_volatility

plt.figure(figsize=(14, 7))
plt.plot(fitted_volatility, label='Gasoline Conditional Volatility', color='blue', alpha=0.7)
plt.twinx()
plt.plot(combined_data['Return_Brent'], label='Brent Returns', color='red', alpha=0.4)
plt.title('Conditional Volatility of Gasoline Returns with Brent Returns')
plt.xlabel('Date')
plt.ylabel('Volatility / Returns')
plt.legend(loc='upper left')
plt.show()

DCC-Garch

In [ ]:
#The general framework is from https://github.com/jack-tobin/mvgarch because it does not exist a standard framework for Multivariate GARCH

garch = UGARCH(order=(1, 1))
garch.spec(returns=gasoline_data["Return"])
garch.fit()

garch_specs = [UGARCH(order=(1, 1)), UGARCH(order=(1, 1))]
dcc = DCCGARCH()
dcc.spec(ugarch_objs=garch_specs, returns=combined_data[["Return_Gasoline", "Return_Brent"]])
dcc.fit()
dcc.plot()